In [1]:
import pandas as pd
import requests
import lxml
from bs4 import BeautifulSoup

# 1. First attempt to scrap sorrywatch.com using the requests package

In [2]:
url = 'https://sorrywatch.com/category/celebrity-apologies/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'}
response = requests.get(url=url, headers=headers)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
print(soup.title)

<title>Celebrity Apologies | SorryWatch</title>


In [7]:
[title.text for title in soup.findAll('h2', attrs={'class':'entry-title'})]

['Apologies of a Jeopardy Jerk',
 'So many afternoons spoiled',
 'Nick Cave on forgiveness',
 'Welp, this is racist and terrible',
 'Bad Apology Bingo, comedian edition!',
 'Apologies for college admissions mishegas']

In [9]:
pages = [1, 2, 3, 4]
titles = []
#for page in pages:
page = 1
u = url+'/page'+str(page)+'/'
response = requests.get(url=u, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
entries = [title.text for title in soup.findAll('h2', attrs={'class':'entry-title'})]
[titles.append(t) for t in entries]
#    for entry in entries:
entry = entries[1]
uu = 'https://sorrywatch.com/'+entry.replace(' ', '-')
rr = requests.get(url=uu, headers=headers)
ss = BeautifulSoup(rr.text, 'html.parser')

In [19]:
'https://sorrywatch.com/'+entry.replace(' ', '-')


'https://sorrywatch.com/So-many-afternoons-spoiled'

In [24]:
u = 'https://news.google.com/search?q=Apologizes%20when%3A1y&hl=en-US&gl=US&ceid=US%3Aen'
response = requests.get(u, headers=headers)
BeautifulSoup(response.text, 'html.parser')
blinks = soup.findAll('article')


In [27]:
params = {
    "q": "apologizes",
    "hl": "en",
    "tbm": "nws",
}
response = requests.get("https://www.google.com/search", headers=headers, params=params)
soup = BeautifulSoup(response.text, 'lxml')


for result in soup.select('.dbsr'):
    title = result.select_one('.nDgy9d').text
    link = result.a['href']
    source = result.select_one('.WF4CUc').text
    snippet = result.select_one('.Y3v8qd').text
    date_published = result.select_one('.WG9SHc span').text
    print(f'{title}\n{link}\n{snippet}\n{date_published}\n{source}\n')
    print('     ')

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

# 2. Not quite what we need. Scrape Google news search results

In [1]:
from serpapi import GoogleSearch

In [9]:
params = {
    "engine": "google",
    "q": "Apologizes for",
    "tbm": "nws",
    "api_key": "<MY GOOGLE API KEY GOES HERE>",
}

search = GoogleSearch(params)
pages = search.pagination()
output = {}
for i, result in enumerate(pages):
    print(f"Current page: {result['serpapi_pagination']['current']}")
    output[result['serpapi_pagination']['current']] = []
    for news_result in result["news_results"]:
        print(f"Title: {news_result['title']}\nLink: {news_result['link']}\n")
        output[result['serpapi_pagination']['current']].append(news_result)


https://serpapi.com/search
Current page: 1
Title: Alabama church apologizes for ‘freak show’ comments by GOP gubernatorial candidate Tim James
Link: https://www.al.com/news/2022/05/alabama-church-apologizes-for-freak-show-comments-by-gop-gubernatorial-candidate-tim-james.html

Title: Lufthansa Apologizes for Blocking Orthodox Jews From Flight
Link: https://www.bloomberg.com/news/articles/2022-05-10/lufthansa-apologizes-for-blocking-orthodox-jews-from-flight

Title: After 800 years, Church of England apologizes to Jews for laws that led to expulsion
Link: https://www.timesofisrael.com/church-of-england-apologizes-to-jews-800-years-after-laws-that-led-to-expulsion/

Title: 'RHOBH's Sutton Stracke Apologizes for 'Insensitive' Reaction to Dorit Kemsley's Home Invasion (Exclusive)
Link: https://www.etonline.com/rhobhs-sutton-stracke-apologizes-for-insensitive-reaction-to-dorit-kemsleys-home-invasion-exclusive

Title: New York Times apologizes for Wordle result with 'unintentional' timelines

In [19]:
# put the results in a Dataframe

import pandas as pd
dat = pd.DataFrame(columns=['position', 'link', 'title', 'source', 'date', 'snippet', 'thumbnail'])
for page in output.keys():
    res = output[page]
    for item in res:
        dat = dat.append(pd.DataFrame([item]), ignore_index=True)


In [34]:
# add a variable for the page number of the results
import numpy as np
results_perPage = 10
dat['page_num'] = np.repeat(list(range(1, int(len(dat)/results_perPage)+1)),results_perPage)

# rearrange columns so page number is first column
cols = dat.columns.tolist()
cols = cols[-1:] + cols[:-1]
dat = dat[cols]

In [37]:
# save a .csv file
dat.to_csv('search_results_5_10_22.csv', sep=',', index=False)